
<img src="./logo_UTN.svg" align="right" width="150" /> 
 Teoría de Circuitos II

# TPL Grupo 1
####  Boirazian Juan - Brunella Nicolas - Pavelek Israel  - Santana Federico - Sureda Nicolas - 

### Objetivos

A partir de un plantilla de filtro, analizar, simular, implementar y medir un circuito que responda a dicha plantilla.

Consolidar los conceptos de teoría moderna de filtrado mediante la implementación circuital.

Medir la respuesta en frecuencia del filtro con un osciloscopio para frecuencias menores entre 20 Hz y 100 kHz.

Descripción

El trabajo práctico consiste en la realización de tres partes que conforman el diseño, análisis, medición y discusión de un filtro activo a partir de una plantilla. Implica la implementación de una función transferencia con un circuito integrado especial, el UAF42 que posee una característica especial: capacitores de muy baja tolerancia (1%) integrados. Esto permite obtener respuestas en frecuencia del filtro sumamente similares a las simulaciones.

La construcción del filtro debe realizarse previo a ir al laboratorio. Las mediciones se espera que se realicen en el laboratorio. Finalmente, la construcción de las gráficas, obtención de respuestas en frecuencia e informe correspondiente a la parte II puede iniciarse en el laboratorio y entregarse hasta dos semanas posterior a la fecha pactada de laboratorio.


Referencias del integrado UAF42:

Página de Texas Instruments con material: https://www.ti.com/product/UAF42

Hoja de datos: https://www.ti.com/lit/ds/symlink/uaf42.pdf 

<table>
  <tr>
    <td>Función de aproximación</td>
    <td>Frecuencia de corte</td>
    <td>Frecuencia de stop</td>
    <td>Atenuación máxima en banda de paso</td>
    <td>Atenuación mínima en banda de stop</td>
  </tr>
  <tr>
    <td>Máxima Planicidad</td>
    <td>1200Hz</td>
    <td>3600Hz</td>
    <td>2dB</td>
    <td>14dB</td>
   </tr>
</table>

Normializando todo en 1200Hz

<table>
  <tr>
    <td>Función de aproximación</td>
    <td>Frecuencia de corte</td>
    <td>Frecuencia de stop</td>
    <td>Atenuación máxima en banda de paso</td>
    <td>Atenuación mínima en banda de stop</td>
  </tr>
  <tr>
    <td>Máxima Planicidad</td>
    <td>1</td>
    <td>3</td>
    <td>2dB</td>
    <td>14dB</td>
   </tr>
</table>

wp=1
ws=3

$\alpha_{max}=2dB$

$\alpha_{min}=14dB$

Caluclo de $\epsilon $

$$ \mid T(\omega)\mid^2=\frac{1}{1+\epsilon^2\omega^{2N}} $$

$$(10^{-\frac{\alpha_{max}}{10}})^2=\frac{1}{1+\epsilon^2}\mid_{\omega=1} $$

$$ 10^{\frac{-\alpha_{max}}{10}} \cdot (1+\epsilon^2)=1 $$ 

$$ \epsilon=\sqrt{10^{\frac{\alpha_{max}}{10}}-1} $$

Calculo de N
<br><br>
$$ 10^{-\frac{\alpha_{min}}{10}}=\frac{1}{1+\epsilon^2 (\omega_s)^{2N}} $$
<br>
$$ 10^{-\frac{\alpha_{min}}{10}} \cdot (1+\epsilon^2 (\omega_s)^{2N}) $$
<br>
$$ (\omega_s)^{2N}=\frac{1-10^{-\frac{\alpha_{min}}{10}}}{10^{-\frac{\alpha_{min}}{10}}\epsilon^2} $$
<br>
$$ N=\frac{log(\frac{10^{(\frac{\alpha_{min}}{10})}-1}{\epsilon^2})}{2log(\omega_s)} $$
<br>


In [ ]:
import numpy as np
import scipy.signal as sig

alfa_max = 2 # dB \alpha_{max} 
alfa_min = 14 # dB \alpha_{min} 
ws=3
    
eps=np.sqrt(10**(alfa_max/10)-1)
N = np.log10(((10**((alfa_min/10))-1)/(eps**2)))/(2*np.log10(ws))
N=np.ceil(N)
eps=np.round(eps,2)    
print('El filtro es de orden:',N,'con Epsilon:',eps)
    

La transferencia del filtro normalizada es:


$$H(s)=\frac{1}{s^2+\sqrt2 s+1}$$



### Desnormalizamos la transferencia:

siendo la frecuencia de desnormalización:

$Wb=0.76^{\frac{-1}{2}}\cdot 2\cdot \pi \cdot 1200Hz$

In [ ]:
from sympy import *
import math 
init_printing(use_unicode=True)
s = symbols('s')
H=(s*s+s*1.41+1)
wb=(1/math.sqrt(0.76))*2*math.pi*1200
H = H.subs(s, s/wb) 
H=monic(H)
H


In [ ]:
H=74801212.30/(s*s+12194.76*s+74801212.30)
H

In [ ]:
from scipy import signal

import matplotlib.pyplot as plt

from splane import *

num=[1]
den=[1,1.41,1]

all_sys=(signal.TransferFunction(num,den))
fig_id, axes_hdl = bodePlot(all_sys, 1, ())
analog_fig_id, analog_axes_hdl = pzmap(all_sys, "", 3, ())

w, h = signal.freqs(num,den)
plt.figure()
plt.plot(w[1:], -np.diff(np.unwrap(np.angle(h)))/np.diff(w))
plt.xscale('log')
plt.title('Filter group delay')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Group delay [seconds]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.show()

### Sintesis del filtro:


<img src="./_UAF42.png" align="center" width="800" /> 

Si tomamos que RF1=RF2=RF

Segun la hoja de datos:

$$\omega_n^2=\frac{1}{R_{F}^2C_1C_2}$$

$$Q=\frac{1+\frac{50K\cdot(R_G+R_Q)}{R_G\cdot R_Q}}{2}$$

$$A=\frac{2}{R_G\cdot (\frac{1}{R_G}+\frac{1}{R_Q}+\frac{1}{50K})}$$

In [ ]:
C=1*10**(-9)
RF=74801212.3*(C**2)
RF=round(1/sqrt(RF),0)
print(f"RF1=RF2={RF:.0f} Ω")
print("Adoptamos RF1=RF2=120KΩ")

In [ ]:
Q=sqrt(74801212.3)/12194.76
RP=((Q*2)-1)/50000
RP=round((2*(1/RP)),0)
print(f"RG=RQ={RP:.0f}Ω")
print("Adoptamos RF1=RF2=220KΩ")


In [ ]:
#La ganancia en continua:
RG=RQ=220000
A=2/(RG*(1/RG+1/RQ+1/50000))
print("Ganacia en continua del filtro:",A)
print("Para que la ganancia sea de 0dB en continua utilizamos un operacional extra que nos provee el UAF42 para ganar 3.2 veces y tener 0dB de ganancia total del filtro")


### Simulación matemática y circuital

<img src="./simu.png" align="center" width="600" /> 